# Few-Shot Prompting Example

This notebook demonstrates **Few-Shot Prompting**, a technique that allows you to "teach" the AI a new task without any model training.

**The Concept:**
Most LLMs are "Zero-Shot" learners, meaning they can answer questions without examples. However, for specific formats, styles, or complex logic, they perform much better if you provide a few examples of **Input → Output** pairs in the prompt. This is called **In-Context Learning**.

**Key Mechanics:**
1.  **Examples**: We provide 3–5 high-quality examples of the task (e.g., "Input: A, Output: B").
2.  **Pattern Recognition**: The model looks at these examples, infers the rule, and applies it to your new input.
3.  **Result**: Better formatting, tone consistency, and accuracy for specialized tasks (like translating to "Pirate English" or extracting data to JSON).

In [ ]:
%pip install openai python-dotenv --quiet

### 1. Setup and Authorization

We start by importing the necessary libraries and loading your OpenAI API key.

In [ ]:
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    api_key = input("Paste your OpenAI API key: ").strip()

client = OpenAI(api_key=api_key)
print("OpenAI client ready! Using gpt-4o-mini (perfect for few-shot tasks)")

### 2. Defining the "Training" Data

In this step, we create the **Few-Shot Prompt**. Unlike a standard instruction ("Translate this"), we feed the model a mini-dataset of correct behaviors.

You can select from several pre-written tasks below to see how the model adapts its behavior based *solely* on the examples provided in the string variable.

In [ ]:
TASKS = {
    "1": "Sentiment Analysis (positive/negative/neutral)",
    "2": "Translate to Pirate English",
    "3": "Extract Named Entities (Person, Org, Location)",
    "4": "Correct Grammar & Make Professional",
    "5": "Answer in JSON Format",
    "6": "Custom (Write your own examples)"
}

print("Few-Shot Prompting Playground")
print("Choose a task to see how examples guide the model:")
for k, v in TASKS.items():
    print(f"  {k}. {v}")

choice = input("\nEnter number (1–6): ").strip()

if choice == "1":
    few_shot_prompt = """
Classify the sentiment as positive, negative, or neutral.

Text: I love this product! It's amazing.
Sentiment: positive

Text: This is the worst experience ever.
Sentiment: negative

Text: It's okay, nothing special.
Sentiment: neutral

Text: The delivery was fast and the quality is good.
Sentiment: positive

Text:"""
    print("Task: Sentiment Analysis")

elif choice == "2":
    few_shot_prompt = """
Translate the following to pirate speech:

Hello, how are you? → Ahoy matey, how be ye?
Thank you very much → Thank ye kindly, ye scurvy dog!
Where is the treasure? → Where be the booty buried?
I'm hungry → Me belly be growlin' fer grub!

Now translate:"""
    print("Task: Pirate English Translator")

elif choice == "3":
    few_shot_prompt = """
Extract named entities and label as PERSON, ORGANIZATION, or LOCATION.

Input: Elon Musk founded SpaceX in California.
Output:
- Elon Musk → PERSON
- SpaceX → ORGANIZATION
- California → LOCATION

Input: Apple is releasing a new iPhone in New York next month.
Output:
- Apple → ORGANIZATION
- iPhone → PRODUCT
- New York → LOCATION

Input:"""
    print("Task: Named Entity Recognition")

elif choice == "4":
    few_shot_prompt = """
Correct grammar and rewrite in professional business tone:

hey wanna grab coffee later? → Hello, would you like to get coffee later this week?
i think its gonna work good → I believe this solution will perform effectively.
sorry im late traffic was bad → Apologies for the delay; I was caught in heavy traffic.

Rewrite:"""
    print("Task: Grammar & Professional Tone")

elif choice == "5":
    few_shot_prompt = """
Answer in valid JSON format only:

What is the capital of France? → {"answer": "Paris", "country": "France"}
Who wrote Romeo and Juliet? → {"answer": "William Shakespeare", "work": "Romeo and Juliet", "year": 1597}
What is 15 + 27? → {"answer": 42, "operation": "addition"}

Question:"""
    print("Task: Force JSON Output")

elif choice == "6":
    print("Custom Few-Shot: Write 3–5 input/output examples below.")
    print("Example format:\nText: hello world\nOutput: HELLO WORLD\n\n")
    few_shot_prompt = input("Paste your full few-shot examples here:\n")
    few_shot_prompt += "\nNow process this input:\n"
else:
    print("Invalid choice. Defaulting to sentiment analysis.")
    choice = "1"
    few_shot_prompt = "Classify sentiment..."  # fallback

### 3. The Execution Engine

Now we run the model.

**How it works:**
1.  **Concatenation**: We take the `few_shot_prompt` (the examples) and append your new `user_input` to the end.
2.  **Completion**: The model sees the pattern (Example 1, Example 2, Example 3, **New Input**) and naturally completes the pattern.
3.  **Low Temperature**: We use a very low temperature (`0.1`) because we want the model to adhere strictly to the format of the examples, not to be creative.

In [ ]:
print("\nFew-Shot Engine Ready! Type 'quit' to exit.\n")

while True:
    user_input = input("Input: ")
    if user_input.strip().lower() in ["quit", "exit", "stop"]:
        print("\nThanks for playing with few-shot!")
        break
    
    if not user_input.strip():
        print("Please enter valid input.")
        continue

    # Combine the examples with the new user input
    full_prompt = few_shot_prompt + " " + user_input

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                # We add a system message to reinforce that it should follow the pattern
                {"role": "system", "content": "You are an expert at following patterns from examples. Never explain, just output in the exact format shown."},
                {"role": "user", "content": full_prompt}
            ],
            temperature=0.1,  # Low temp = high consistency
            max_tokens=500
        )

        result = response.choices[0].message.content.strip()
        print(f"\nOutput:\n{result}\n\n" + "─"*50)
        
    except Exception as e:
        print(f"\nError: {e}\n")